In [6]:
import pandas as pd
import numpy as np
import os

output_dir = r"D:\71 PsychLLM\outs"
tom_df = pd.read_csv(os.path.join(output_dir, 'fantom','consolidated.csv'))
mpi_df = pd.read_csv(os.path.join(output_dir, 'personality','consolidated.csv'))

In [7]:
tom_df.columns

Index(['model', 'conversation_input_type', 'inaccessible:set:ALL*',
       'inaccessible:set:ALL', 'inaccessible:belief:multiple-choice',
       'inaccessible:answerability:binary-f1',
       'inaccessible:info_accessibility:binary-f1',
       'inaccessible:tom:binary:wrong_reasons:freq', 'filename', 'personality',
       'inaccessible:belief:distance', 'inaccessible:belief_true_word-f1',
       'inaccessible:answerability:set:ALL', 'inaccessible:answerability:list',
       'inaccessible:info_accessibility:set:ALL',
       'inaccessible:info_accessibility:list', 'fact_word-f1',
       'inaccessible:tom:lists:wrong_reasons:freq', 'inaccessible:first-order',
       'inaccessible:second-order', 'inaccessible:second-order:acyclic',
       'inaccessible:second-order:cyclic', 'inaccessible:set:ALL_character',
       'inaccessible:set:character_answer_consistency'],
      dtype='object')

In [8]:
mpi_df['model'].unique()

array(['falcon-7b-instruct', 'gpt-3.5-turbo-1106', 'Llama-2-7b-chat-hf',
       'mistral-instruct', 'zephyr-7b-beta'], dtype=object)

In [9]:
import pandas as pd
import statsmodels.api as sm


models = ['Llama-2-7b-chat-hf', 'mistral-instruct','falcon-7b-instruct', 'zephyr-7b-beta', 'gpt-3.5-turbo-1106']
personalities = [
    'Agreeableness', 'Conscientiousness', 'Extraversion', 'Machiavellianism', 
    'Narcissism', 'Neuroticism', 'Openness','Psychopathy', 'Task-Specific'
]
focus_mpis = ['A_mean', 'C_mean', 'E_mean','N_mean', 'O_mean']
base_personality = 'None'

for focus_task in ['inaccessible:belief:multiple-choice','inaccessible:answerability:binary-f1','inaccessible:info_accessibility:binary-f1']:

    ##### Format table ######
    print(focus_task)
    
    output = []
    for focus_model in models:

        for focus_personality in personalities:

            base_score = tom_df[(tom_df['model']==focus_model) & (tom_df['personality']==base_personality)][focus_task].values[0]
            score = tom_df[(tom_df['model']==focus_model) & (tom_df['personality']==focus_personality)][focus_task].values[0]
            diff_tom = score-base_score
            mpi_scores = []
            for mpi in focus_mpis:
                base_score = mpi_df[(mpi_df['model']==focus_model) & (mpi_df['personality']==base_personality)][mpi].values[0]
                score = mpi_df[(mpi_df['model']==focus_model) & (mpi_df['personality']==focus_personality)][mpi].values[0]
                mpi_scores.append(score-base_score)

            output.append([focus_model,focus_personality,diff_tom]+mpi_scores)

    output = pd.DataFrame(output, columns=['model','personality','tom_diff']+focus_mpis)
    
    ##### Regressions ######
    df = pd.get_dummies(output, columns=['model'], drop_first=True)
    X = df.drop(['tom_diff','personality'], axis=1)  # Independent variables
    Y = df['tom_diff']               # Dependent variable
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    print(model.summary())


inaccessible:belief:multiple-choice
                            OLS Regression Results                            
Dep. Variable:               tom_diff   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.519
Method:                 Least Squares   F-statistic:                     6.269
Date:                Sat, 10 Feb 2024   Prob (F-statistic):           3.08e-05
Time:                        20:37:02   Log-Likelihood:                -59.477
No. Observations:                  45   AIC:                             139.0
Df Residuals:                      35   BIC:                             157.0
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------